## Getting started with TensorFlow - I

_Notes compiled from Chapter 9: Hands-On Machine Learning with Scikit-Learn and TensorFlow_

We start by creating a simple graph in Tensor Flow.

In [3]:
import tensorflow as tf

In [25]:
def simple_graph():
    
    # creating the computational graph. Variables are not initialized, and no computation is performed yet.
    x = tf.Variable(3, name="x")
    y = tf.Variable(4, name="y")
    f = x*x*y + y + 2

    # To evaluate the graph constructed above, we open a tensorflow session and use it to initialize variables and evaluate f.
    with tf.Session() as sess:
        x.initializer.run()
        y.initializer.run()
        result = f.eval()
    
    return

simple_graph()

A TensorFlow session takes care of placing the operations onto devices(CPUs and GPUs) and running them, and it holds all the variable values.

NOTE: Calling x.initializer.run() is equivalent to calling tf.get_default_session().run(x.initializer).

***

###  Instead of manually running the initializer for every single variable, we can use the global_variables_initializer() as follows:

In [26]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

***

### In Tensorflow, any node that is created is automatically added to the default graph.

In [27]:
x1 = tf.Variable(2)
x1.graph is tf.get_default_graph()

True

***

### Attributes and methods of the default graph:

In [28]:
print(dir(tf.get_default_graph()))

['_ControlDependenciesController', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_device_to_stack', '_add_function', '_add_new_tf_operations', '_add_op', '_apply_device_functions', '_as_graph_def', '_as_graph_element_locked', '_attr_scope', '_attr_scope_map', '_building_function', '_c_graph', '_check_not_finalized', '_collections', '_colocate_with_for_gradient', '_colocation_stack', '_container', '_control_dependencies_for_inputs', '_control_dependencies_stack', '_control_flow_context', '_copy_functions_to_graph_def', '_create_op_from_tf_operation', '_create_op_helper', '_current_control_dependencies', '_default_original_op', '_device_function_stack', '_device_functions_outer_to_inner', '_distribution_strateg

In [29]:
tf.get_default_graph().get_operations()

[<tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'x/initial_value' type=Const>,
 <tf.Operation 'x' type=VariableV2>,
 <tf.Operation 'x/Assign' type=Assign>,
 <tf.Operation 'x/read' type=Identity>,
 <tf.Operation 'y/initial_value' type=Const>,
 <tf.Operation 'y' type=VariableV2>,
 <tf.Operation 'y/Assign' type=Assign>,
 <tf.Operation 'y/read' type=Identity>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'mul_1' type=Mul>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=Add>,
 <tf.Operation 'x_1/initial_value' type=Const>,
 <tf.Operation 'x_1' type=VariableV2>,
 <tf.Operation 'x_1/Assign' type=Assign>,
 <tf.Operation 'x_1/read' type=Identity>,
 <tf.Operation 'y_1/initial_value' type=Const>,
 <tf.Operation 'y_1' type=VariableV2>,
 <tf.Operation 'y_1/Assign' type=Assign>,
 <tf.Operatio

#### Running the same commands in jupyter notebook leads to addition of duplicate nodes in the default graph as we can see above. A conveninet solution to this is to <font color=blue>reset the default graph</font> as follows:

In [30]:
tf.reset_default_graph()
tf.get_default_graph().get_operations()

[]

***

### Lifecycle of a Node Value

In [32]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


When we evaluate a node in tensorflow, it automatically evaluates its dependencies first. 

In the above snippet, we define a simple graph consisting of w, x, y, and z nodes. Then we start the session and run the graph to evaluate node y. TensorFlow detects that node y is dependent on x, which in turn depends on w. So the order of evaluation is w, then x and then y.

However, to evaluate z, it again evaluates x, and w. It does not reuse the previous evaluation of w and x.

To evaluate y and z efficiently, without evaluating w and x twice, we evaluate w and x in just one graph run as shown below:

In [34]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In a single process TensorFlow, multiple sessions do not share any state, even if they reuse the same graph. Each session has its own copy of every variable. A variable starts its life when its initializer is run, and it ends when the session is closed.

***

### TensorFlow Operations

TensorFlow operations can take any number of inputs and produce any number of outpts. Constants and variables take no input, and they are called _source operations_.